In [2]:
import re
import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import one_hot
from sklearn.metrics import confusion_matrix,accuracy_score
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding,LSTM,Dense,Dropout

print('Import successful')

Import successful


In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/max/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
fake_news_df = pd.read_csv('./data/Fake.csv')
fake_news_df.dropna(inplace=True)
fake_news_df['label'] = 0
fake_news_df.head(10)

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [5]:
true_news_df = pd.read_csv('./data/True.csv')
true_news_df.dropna(inplace=True)
true_news_df.reset_index(inplace=True)
true_news_df['label'] = 1
true_news_df['index'] = true_news_df['index'] + fake_news_df['title'].size
true_news_df.set_index('index', inplace=True)
true_news_df.head(10)

,title,text,subject,date,label
index,,,,,
23481,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
23482,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
23483,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
23484,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
23485,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1
23486,"White House, Congress prepare for talks on spe...","WEST PALM BEACH, Fla./WASHINGTON (Reuters) - T...",politicsNews,"December 29, 2017",1
23487,"Trump says Russia probe will be fair, but time...","WEST PALM BEACH, Fla (Reuters) - President Don...",politicsNews,"December 29, 2017",1
23488,Factbox: Trump on Twitter (Dec 29) - Approval ...,The following statements were posted to the ve...,politicsNews,"December 29, 2017",1
23489,Trump on Twitter (Dec 28) - Global Warming,The following statements were posted to the ve...,politicsNews,"December 29, 2017",1


In [6]:
df = pd.concat([fake_news_df, true_news_df])
df.iloc[23479:23483]

,title,text,subject,date,label
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",0
23481,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
23482,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1


In [9]:
X = df['title'].astype('str')
y = df['label']
print('Split to X and y')
print(X[23479:23486])
print(y[23479:23486])

Split to X and y
23479    How to Blow $700 Million: Al Jazeera America F...
23480    10 U.S. Navy Sailors Held by Iranian Military ...
23481    As U.S. budget fight looms, Republicans flip t...
23482    U.S. military to accept transgender recruits o...
23483    Senior U.S. Republican senator: 'Let Mr. Muell...
23484    FBI Russia probe helped by Australian diplomat...
23485    Trump wants Postal Service to charge 'much mor...
Name: title, dtype: object
23479    0
23480    0
23481    1
23482    1
23483    1
23484    1
23485    1
Name: label, dtype: int64


In [10]:
ps = PorterStemmer()
corpus = []
for i in range(len(X)):
    text = X[i]
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    text = [ps.stem(t) for t in text if t not in stopwords.words('english')]
    corpus.append(' '.join(text))
print('Done.')

Done.


In [11]:
vocab_size = 5000
sent_len = 20
one_hot_encoded = [one_hot(x, vocab_size) for x in corpus]
one_hot_encoded = pad_sequences(one_hot_encoded, maxlen=sent_len)
one_hot_encoded[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
        711, 4073, 4987,  750, 4656, 4363,  684, 4438, 4322], dtype=int32)

In [19]:
X = np.array(one_hot_encoded)
y = np.array(y)
X_train, X_rem, y_train, y_rem = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_rem, y_rem, test_size=0.5, random_state=42)
print(f'Split is done correctly: {X_train.size + X_val.size + X_test.size == X.size}')

Split is done correctly: True


In [39]:
no_of_output_features = 40

model = Sequential()
model.add(Embedding(vocab_size, no_of_output_features, input_length=sent_len))
model.add(LSTM(units=100, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_5 (LSTM)               (None, 100)               56400     
                                                                 
 dense_6 (Dense)             (None, 128)               12928     
                                                                 
 dropout_6 (Dropout)         (None, 128)               0         
                                                                 
 dense_7 (Dense)             (None, 1)                 129       
                                                                 
Total params: 269,457
Trainable params: 269,457
Non-trainable params: 0
_________________________________________________________________


In [40]:
history = model.fit(X_train,y_train,validation_data=(X_val,y_val),batch_size=64,epochs=40, callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4)])

Epoch 1/40
421/421 [==============================] - 11s 24ms/step - loss: 0.2380 - accuracy: 0.9013 - val_loss: 0.1679 - val_accuracy: 0.9308
Epoch 2/40
421/421 [==============================] - 10s 24ms/step - loss: 0.1221 - accuracy: 0.9533 - val_loss: 0.1552 - val_accuracy: 0.9385
Epoch 3/40
421/421 [==============================] - 21s 49ms/step - loss: 0.0906 - accuracy: 0.9669 - val_loss: 0.1722 - val_accuracy: 0.9372
Epoch 4/40
421/421 [==============================] - 23s 55ms/step - loss: 0.0676 - accuracy: 0.9762 - val_loss: 0.1734 - val_accuracy: 0.9374
Epoch 5/40
421/421 [==============================] - 10s 24ms/step - loss: 0.0515 - accuracy: 0.9820 - val_loss: 0.2045 - val_accuracy: 0.9360
Epoch 6/40
421/421 [==============================] - 11s 25ms/step - loss: 0.0410 - accuracy: 0.9861 - val_loss: 0.2350 - val_accuracy: 0.9333


In [41]:
pred = [0 if p < 0.5 else 1 for p in model.predict(X_test)]
confusion_matrix(y_test, pred)

array([[4504,  239],
       [ 335, 3902]])

In [43]:
accuracy_score(y_test,pred)

0.9360801781737194

In [45]:
model.save_weights('./models/fake_news_v1/model')